In [ ]:
!pip install yfinance pandas

In [ ]:
import yfinance as yf
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
import time
import pytz

# Configuración
DB_PATH = "financial_data.db"
TICKERS = ["AAPL", "MSFT", "GOOGL"]
INTERVAL = "1m"
TIMEZONE = pytz.timezone("America/Argentina/Buenos_Aires")  # ART (-03:00)

# Conexión a SQLite
def connect_db():
    return sqlite3.connect(DB_PATH)

# Crear la tabla si no existe
def create_table():
    conn = connect_db()
    c = conn.cursor()
    c.execute("""
        CREATE TABLE IF NOT EXISTS ohlcv_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            ticker TEXT NOT NULL,
            datetime TEXT NOT NULL,
            open REAL NOT NULL,
            high REAL NOT NULL,
            low REAL NOT NULL,
            close REAL NOT NULL,
            volume INTEGER NOT NULL,
            interval TEXT NOT NULL,
            UNIQUE(ticker, datetime, interval)
        )
    """)
    conn.commit()
    conn.close()

# Obtener la última fecha registrada para un ticker
def get_last_datetime(ticker, interval=INTERVAL):
    conn = connect_db()
    c = conn.cursor()
    c.execute("""
        SELECT MAX(datetime) FROM ohlcv_data WHERE ticker = ? AND interval = ?
    """, (ticker, interval))
    result = c.fetchone()[0]
    conn.close()
    return result if result else None

# Descargar datos históricos de yfinance
def fetch_historical_data(ticker, start_date, end_date, interval=INTERVAL):
    try:
        stock = yf.Ticker(ticker)
        df = stock.history(start=start_date, end=end_date, interval=interval)
        if not df.empty:
            # Mantener Datetime como índice
            df['ticker'] = ticker
            df['interval'] = interval
            # Renombrar columnas directamente
            df = df.rename(columns={
                'Open': 'open',
                'High': 'high',
                'Low': 'low',
                'Close': 'close',
                'Volume': 'volume'
            })
            # Seleccionar columnas necesarias (Datetime sigue siendo índice)
            df = df[['ticker', 'open', 'high', 'low', 'close', 'volume', 'interval']]
            print(f"Datos procesados para {ticker}:\n{df.head()}\n")  # Para depuración
            return df
        else:
            print(f"No se encontraron datos para {ticker}")
            return None
    except Exception as e:
        print(f"Error al descargar {ticker}: {e}")
        return None

# Guardar datos en SQLite
def save_to_db(df):
    conn = connect_db()
    c = conn.cursor()
    # Iterar sobre el DataFrame usando el índice Datetime
    for datetime_index, row in df.iterrows():
        c.execute("""
            INSERT OR IGNORE INTO ohlcv_data (ticker, datetime, open, high, low, close, volume, interval)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            row['ticker'],
            datetime_index.isoformat(),  # Extraer Datetime del índice
            row['open'],
            row['high'],
            row['low'],
            row['close'],
            int(row['volume']),
            row['interval']
        ))
    conn.commit()
    conn.close()

# Proceso diario ajustado
def daily_update():
    create_table()
    now = datetime.now(TIMEZONE)
    # Si es fin de semana, ajustamos al último viernes
    if now.weekday() >= 5:  # Sábado (5) o domingo (6)
        days_to_subtract = now.weekday() - 4  # Retrocede al viernes
        end_date = now - timedelta(days=days_to_subtract)
    else:
        end_date = now - timedelta(days=1)  # Último día completo

    end_date = end_date.replace(hour=23, minute=59, second=59, microsecond=0)
    start_date = end_date - timedelta(days=1)  # Solo 1 día de datos

    for ticker in TICKERS:
        last_datetime = get_last_datetime(ticker)
        if last_datetime:
            last_dt = datetime.fromisoformat(last_datetime)
            if last_dt >= end_date:
                print(f"No hay datos nuevos para {ticker}.")
                continue
            start_date = last_dt + timedelta(minutes=1)

        print(f"Actualizando datos para {ticker} desde {start_date} hasta {end_date}...")
        df = fetch_historical_data(ticker, start_date, end_date)
        if df is not None:
            save_to_db(df)
            print(f"Datos de {ticker} actualizados.")
        time.sleep(2)

# Descargar historial completo (máximo 7 días para 1m)
def initial_load():
    create_table()
    now = datetime.now(TIMEZONE)
    end_date = now.replace(hour=23, minute=59, second=59, microsecond=0)
    start_date = end_date - timedelta(days=DAYS_BACK)

    for ticker in TICKERS:
        print(f"Descargando historial para {ticker} desde {start_date} hasta {end_date}...")
        # Dividir en bloques de 7 días para mayor estabilidad
        current_start = start_date
        while current_start < end_date:
            current_end = min(current_start + timedelta(days=DAYS_BACK), end_date)
            df = fetch_historical_data(ticker, current_start, current_end)
            if df is not None:
                save_to_db(df)
                print(f"Datos históricos de {ticker} guardados para {current_start} a {current_end}.")
            current_start = current_end
            time.sleep(2)  # Pausa para evitar límites de la API

if __name__ == "__main__":
    daily_update()

Actualizando datos para AAPL desde 2025-04-03 23:59:59-03:00 hasta 2025-04-04 23:59:59-03:00...
Datos procesados para AAPL:
                          ticker        open        high         low  \
Datetime                                                               
2025-04-04 09:30:00-04:00   AAPL  193.925003  196.250000  192.800003   
2025-04-04 09:31:00-04:00   AAPL  196.155197  196.578903  195.163605   
2025-04-04 09:32:00-04:00   AAPL  195.929993  196.360001  195.279999   
2025-04-04 09:33:00-04:00   AAPL  196.270004  197.169998  195.425705   
2025-04-04 09:34:00-04:00   AAPL  195.434998  196.845001  195.139999   

                                close   volume interval  
Datetime                                                 
2025-04-04 09:30:00-04:00  196.169998  9520091       1m  
2025-04-04 09:31:00-04:00  195.949997   723579       1m  
2025-04-04 09:32:00-04:00  196.309998   863846       1m  
2025-04-04 09:33:00-04:00  195.434998   836390       1m  
2025-04-04 09:34:00-04:

In [ ]:
initial_load()

Descargando datos históricos para AAPL...
Datos procesados para AAPL:
                          ticker        open        high         low  \
Datetime                                                               
2025-03-31 09:30:00-04:00   AAPL  217.220001  218.500000  216.830002   
2025-03-31 09:31:00-04:00   AAPL  218.054993  218.309998  217.509995   
2025-03-31 09:32:00-04:00   AAPL  217.729996  218.039993  217.127502   
2025-03-31 09:33:00-04:00   AAPL  217.229996  217.570007  216.959595   
2025-03-31 09:34:00-04:00   AAPL  217.149994  217.419998  216.800003   

                                close   volume interval  
Datetime                                                 
2025-03-31 09:30:00-04:00  218.100006  2701302       1m  
2025-03-31 09:31:00-04:00  217.720001   399855       1m  
2025-03-31 09:32:00-04:00  217.240005   492161       1m  
2025-03-31 09:33:00-04:00  217.125000   480012       1m  
2025-03-31 09:34:00-04:00  217.127502   474075       1m  

Datos históricos d